In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest
#sys.path.append('C:\\Users\\brittany\\henderson\\GitHub\\GBM_for_CPTAC\\')
#import cis_functions as f
#sys.path.append(R'C:\Users\Lindsey\Documents\GitHub\WhenMutationsDontMatter\\')
#import plot_utils as p


import cptac
import cptac.utils as u

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#load GBM data 
#cptac.download(dataset='gbm')
brain = cptac.Gbm()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 3)


In [8]:
lipids = brain.get_lipidomics()
lipids.filter(like='PIP', axis=1)

Name
Patient_ID
C3L-00365
C3L-00674
C3L-00677
C3L-01043
C3L-01045
...
PT-Q2AG.N
PT-R55F.N
PT-RN5K.N


In [3]:
#filter for genes only interacting genes.
df1 = brain.join_omics_to_mutations(omics_df_name="proteomics", mutations_genes="EGFR")
df1_tumor = df1.loc[df1['Sample_Status'] == "Tumor"]
df1_tumor = df1_tumor.drop(columns=['EGFR_Mutation', 'EGFR_Location', "EGFR_Mutation_Status"])
df1_tumor

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 93 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 2)


Name,A1BG_proteomics,A2M_proteomics,AAAS_proteomics,AACS_proteomics,AADAT_proteomics,AAED1_proteomics,AAGAB_proteomics,AAK1_proteomics,AAMDC_proteomics,AAMP_proteomics,...,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,Sample_Status
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00104,0.077630,0.487228,-0.254208,-0.144373,0.551881,-0.025276,-0.467451,-0.089511,-0.078806,0.329603,...,-0.047437,-0.105908,-0.347076,NaN,0.459635,0.079452,-0.784983,-0.488441,0.167990,Tumor
C3L-00365,-0.145975,0.798796,0.184242,-0.470603,NaN,0.390211,0.245466,-0.609998,0.118625,-0.086927,...,0.161975,-0.213093,0.235571,NaN,0.107421,0.048724,0.138403,-0.290141,0.405037,Tumor
C3L-00674,0.821991,1.096470,-0.094421,-0.106304,0.084578,0.176402,-0.248151,0.014061,-0.699773,-0.638462,...,-0.065534,-0.306717,0.879991,NaN,0.883564,-0.172222,0.011876,-0.131889,-0.503581,Tumor
C3L-00677,-0.064567,0.129385,0.047751,-0.118187,0.237434,NaN,0.303847,0.322163,-0.555479,-0.363414,...,-0.254535,0.463653,0.580230,0.503044,-0.604986,0.178077,-0.720059,-0.150197,-0.268715,Tumor
C3L-01040,-0.763691,-1.031834,-0.217194,-0.695701,0.184173,-0.474816,-0.051789,0.344842,-0.642746,0.068863,...,-0.092502,0.010639,-0.465079,NaN,-0.500083,0.112651,1.004660,-0.230304,-0.102416,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-03183,0.230843,0.426624,0.420155,0.303960,0.078341,0.833128,0.205727,-0.434020,0.151202,0.208972,...,-0.021466,-0.049569,0.128461,-0.437797,-0.144343,-0.024981,-0.275897,-0.116367,0.351479,Tumor
C3N-03184,-0.635316,-0.478657,0.647004,-0.050322,-0.440517,0.092256,-0.006924,-0.511331,0.101877,0.128163,...,-0.034616,0.158269,NaN,NaN,0.906358,0.287317,0.745880,-0.320613,-0.180511,Tumor
C3N-03186,0.616640,0.767029,-0.231897,0.050841,0.304359,-0.384608,-0.205284,0.513575,0.151769,-0.307037,...,-0.110846,-0.069605,-0.183921,NaN,-0.681170,0.095601,-0.096716,0.049811,-0.409252,Tumor


In [15]:
# EGF, TGFA, HBEGF, and BTC are thought to be high-affinity ligands, whereas AREG, EREG, and EPGN
#GBM HBEGF, 
df1_tumor.filter(like='BTC', axis=1)

Name
Patient_ID
C3L-00104
C3L-00365
C3L-00674
C3L-00677
C3L-01040
...
C3N-03183
C3N-03184
C3N-03186
